In [0]:
!python mlip_setup_env.py

     |████████████████████████████████| 1.4MB 2.7MB/s 
  Found existing installation: wandb 0.8.26
    Uninstalling wandb-0.8.26:
      Successfully uninstalled wandb-0.8.26
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


In [0]:
%tensorflow_version 2.x

import numpy as np
import pandas as pd

TensorFlow 2.x selected.


# Perform preprocessing explicitly (optional)

## Write to parquet (slow)

In [0]:
from preprocessing import perform_preprocessing

preprocess_args = dict(
    image_width=64,
    image_height=64,
    padding=6,
)

# requires a (virtual) machine with at least 12 GB of RAM
# on physical machines I expect the SSD to take over from
# RAM when necessary
perform_preprocessing(preprocess_args, out='parquet')

## Write to png files (less slow)

In [0]:
from preprocessing import perform_preprocessing

preprocess_args = dict(
    image_width=64,
    image_height=64,
    padding=6,
)

# requires less RAM than parquet method, about ~4 GB
# also this method seems faster, because here
# the time required to save the result is included
# in the time below, while above it is only the conversion time.
perform_preprocessing(preprocess_args, out='png')

# New code in parts (for insight/interaction)
To determine the effect of changing the region of interest, a plan is developed to test it. The data preprocessing automatically selects a square box that contains the sign completely, this is called the region of interest(ROI). Then this ROI is, depending on the size of the automatic selection, either downscaled or upscaled to a different image size. Most ROI's will be upscaled since the original images were 136x237 and only a small fraction of the images actually contained a sign. 

The square ROI is varied from 32x32 (downscaling),64x64,96x96,128x128,160x160(upscaling) to determine the effects scaling the ROI. With this range, the required padding is computed with \cref{eq:padding} to prevent cutoffs when the image is rotated and sheared.
\begin{equation}
  \frac{side}{2}tan(rotation+shear)
  \label{eq:padding}
\end{equation}
To ensure the planned evaluation strategy is feasible the computational time is estimated before engaging the run to balance cost-result performance. 

In [0]:
from helper import train_from_prep

datagen_args = dict(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=8,  # randomly rotate images in the range (degrees, 0 to 180)
    zoom_range = 0.15, # Randomly zoom image 
    width_shift_range=0.15,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.15,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=False,  # randomly flip images
    vertical_flip=False,
    shear_range=8, #Shear Intensity (Shear angle in counter-clockwise direction in degrees)
)


preprocess_args = dict(
    image_width = 64,
    image_height = 64,
    padding = 6,
)

model = train_from_prep(datagen_args, name='Image 64, padding 6', 
                        preprocess_args=preprocess_args)

model.save("example.h5")

Converting grapheme_root to one-hot encoding
Converting vowel_diacritic to one-hot encoding
Converting consonant_diacritic to one-hot encoding
Found 184772 validated image filenames.
Found 16068 validated image filenames.
  {'out_root': '...', 'out_vowel': '...', 'out_consonant': '...'}
    to  
  ['...', '...', '...']
  {'out_root': '...', 'out_vowel': '...', 'out_consonant': '...'}
    to  
  ['...', '...', '...']
Train for 721 steps, validate for 62 steps
Epoch 1/30
721/721 [==============================] - 144s 199ms/step - loss: 10.2637 - out_root_loss: 4.0971 - out_vowel_loss: 1.1138 - out_consonant_loss: 0.9557 - out_root_acc: 0.0790 - out_vowel_acc: 0.6074 - out_consonant_acc: 0.6792 - val_loss: 6.6041 - val_out_root_loss: 2.7268 - val_out_vowel_loss: 0.5807 - val_out_consonant_loss: 0.5698 - val_out_root_acc: 0.2588 - val_out_vowel_acc: 0.7948 - val_out_consonant_acc: 0.7921
Epoch 2/30
721/721 [==============================] - 135s 187ms/step - loss: 5.0409 - out_root_loss: 

KeyboardInterrupt: ignored

In [0]:
from starter_eda_model_funcs import get_model
model = get_model()

In [0]:
import os
data_path = 'Data/'; prep_folder = 'prep/'
# set variables and load training labels
image_width, image_height = preprocess_args['image_width'], preprocess_args['image_height']
if image_width == image_height:
    preprocess_args['image_size'] = image_width
prep_path = data_path + prep_folder

# check if preprocessing has been done, and coincides with current arguments
if os.path.exists(prep_path + 'config.csv'):
    # check preprocessing arguments, and convert from DataFrame to Series
    prep_config = pd.read_csv(prep_path + 'config.csv', index_col=0, header=None).iloc[:,0]
    if not prep_config.equals(pd.Series(preprocess_args)):
        # preprocessing arguments do not coince, so
        # preprocessing needs to be done with new arguments
        print("Preprocessing arguments changed. Performing data preprocessing...")
        perform_preprocessing(preprocess_args, data_path=data_path, prep_folder=prep_folder, out='png')
else:
    print("Performing data preprocessing...")
    perform_preprocessing(preprocess_args, data_path=data_path, prep_folder=prep_folder, out='png')


# read train labels
train_df_ = pd.read_csv(f'{data_path}/train.csv')

In [0]:
if model == None:
    assert image_width == image_height, "function get_model not yet ready for rectanglurar images"
    model = get_model(img_size=image_width)

In [0]:
from helper import to_one_hot

# add filename column to train labels df
train_df_['filename'] = train_df_['image_id'] + '.png'

# convert target labels to one-hot encoding
# this also returns the ordered labels of the newly created columns
train_df_, features = to_one_hot(train_df_, one_hot_columns = ['grapheme_root', 'vowel_diacritic', 'consonant_diacritic'])
assert len(features) == 168 + 11 + 7, print(f"found {len(features)} one-hot encoded features")

Converting grapheme_root to one-hot encoding
Converting vowel_diacritic to one-hot encoding
Converting consonant_diacritic to one-hot encoding


In [0]:
from starter_eda_model_funcs import MultiOutputDataGenerator
# define data augmentation generator for multiple outputs
train_datagen = MultiOutputDataGenerator(**datagen_args)
val_datagen = MultiOutputDataGenerator({})

In [0]:
from sklearn.model_selection import train_test_split
from helper import flow_from_prep
batch_size = 256
# split the train and validation data
train_df, val_df = train_test_split(train_df_, test_size=0.08, random_state=666)

# couple the data generator to the prepared images
train_generator = flow_from_prep(train_datagen, df=train_df, prep_path=prep_path, labels=features,
                                image_size=(image_width, image_height), batch_size=batch_size)
val_generator = flow_from_prep(val_datagen, df=val_df, prep_path=prep_path, labels=features,
                                image_size=(image_width, image_height), batch_size=batch_size)

Found 184772 validated image filenames.
Found 16068 validated image filenames.


In [0]:
from helper import GlobalAccuracyCallback, generator_wrapper
# create custom global accuracy with weights 50%, 25%, 25%
global_accuracy_callback = GlobalAccuracyCallback(validation_generator = val_generator)

In [0]:
from helper import get_lr_reduction_calbacks
epochs = 1
# get lr reduction on plateau callbacks
lr_reduction_root, lr_reduction_vowel, lr_reduction_consonant = get_lr_reduction_calbacks()

# Fit the model
# wrap the data generator to support multiple output labels
history = model.fit(generator_wrapper(train_generator), validation_data = generator_wrapper(val_generator),
                    epochs = epochs, steps_per_epoch=train_generator.n//train_generator.batch_size,
                    validation_steps=val_generator.n//val_generator.batch_size,
                    callbacks=[lr_reduction_root, lr_reduction_vowel, lr_reduction_consonant,
                                global_accuracy_callback])

  {'out_root': '...', 'out_vowel': '...', 'out_consonant': '...'}
    to  
  ['...', '...', '...']
  {'out_root': '...', 'out_vowel': '...', 'out_consonant': '...'}
    to  
  ['...', '...', '...']
Train for 721 steps, validate for 62 steps
721/721 [==============================] - 136s 188ms/step - loss: 10.0076 - out_root_loss: 3.9592 - out_vowel_loss: 1.1304 - out_consonant_loss: 0.9589 - out_root_acc: 0.0899 - out_vowel_acc: 0.5985 - out_consonant_acc: 0.6760 - val_loss: 6.0754 - val_out_root_loss: 2.5231 - val_out_vowel_loss: 0.5551 - val_out_consonant_loss: 0.4741 - val_out_root_acc: 0.2718 - val_out_vowel_acc: 0.8012 - val_out_consonant_acc: 0.8245


In [0]:
val_generator.reset()
metrics = model.evaluate(generator_wrapper(val_generator), 
                         steps=val_generator.n // val_generator.batch_size,
                         verbose=1)
metric_labels = model.metrics_names

  {'out_root': '...', 'out_vowel': '...', 'out_consonant': '...'}
    to  
  ['...', '...', '...']
62/62 [==============================] - 3s 49ms/step - loss: 6.0742 - out_root_loss: 2.5211 - out_vowel_loss: 0.5526 - out_consonant_loss: 0.4793 - out_root_acc: 0.2697 - out_vowel_acc: 0.8026 - out_consonant_acc: 0.8212


In [0]:
global_acc = 0
global_weights = {'out_root_acc': 0.5, 'out_vowel_acc': 0.25, 'out_consonant_acc': 0.25}
assert np.sum(list(global_weights.values())) == 1., "Set weights to sum to one, for normalization"

In [0]:
metrics, metric_labels

([6.074159445301179,
  2.5210865,
  0.5526414,
  0.47934666,
  0.26972026,
  0.8026084,
  0.8211945],
 ['loss',
  'out_root_loss',
  'out_vowel_loss',
  'out_consonant_loss',
  'out_root_acc',
  'out_vowel_acc',
  'out_consonant_acc'])

In [0]:
for key, weight in global_weights.items():
    idx = metric_labels.index(key)
    global_acc += metrics[idx] * weight

In [0]:
global_acc

0.5408108532428741

# Utilize new preprocessing

In [0]:
from helper import train_from_prep

datagen_args = dict(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=8,  # randomly rotate images in the range (degrees, 0 to 180)
    zoom_range = 0.15, # Randomly zoom image 
    width_shift_range=0.15,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.15,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=False,  # randomly flip images
    vertical_flip=False,
    shear_range=8, #Shear Intensity (Shear angle in counter-clockwise direction in degrees)
)

# leaves actual grapheme at 64x64, but adds a padding of 16 all around
preprocess_args = dict(
    image_width = 96,
    image_height = 96,
    padding = 16,
)

model = train_from_prep(datagen_args, name='Image 96, padding 16', 
                        preprocess_args=preprocess_args)

Performing data preprocessing...


Converting grapheme_root to one-hot encoding
Converting vowel_diacritic to one-hot encoding
Converting consonant_diacritic to one-hot encoding
Found 184772 validated image filenames.
Found 16068 validated image filenames.
  {'out_root': '...', 'out_vowel': '...', 'out_consonant': '...'}
    to  
  ['...', '...', '...']
  {'out_root': '...', 'out_vowel': '...', 'out_consonant': '...'}
    to  
  ['...', '...', '...']
Train for 721 steps, validate for 62 steps
Epoch 1/30
720/721 [============================>.] - ETA: 0s - loss: 2.5178 - out_root_loss: 4.0038 - out_vowel_loss: 1.1792 - out_consonant_loss: 0.8843 - out_root_acc: 0.0874 - out_vowel_acc: 0.5843 - out_consonant_acc: 0.7035WARNING:tensorflow:sample_weight modes were coerced from
  {'out_root': '...', 'out_vowel': '...', 'out_consonant': '...'}
    to  
  ['...', '...', '...']
721/721 [==============================] - 220s 305ms/step - loss: 2.5168 - out_root_loss: 4.0024 - out_vowel_loss: 1.1785 - out_consonant_loss: 0.8840 

NameError: ignored

In [0]:
from helper import train_from_prep

datagen_args = dict(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=8,  # randomly rotate images in the range (degrees, 0 to 180)
    zoom_range = 0.15, # Randomly zoom image 
    width_shift_range=0.15,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.15,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=False,  # randomly flip images
    vertical_flip=False,
    shear_range=8, #Shear Intensity (Shear angle in counter-clockwise direction in degrees)
)

# leaves actual grapheme at 64x64, but adds a padding of 16 all around
preprocess_args = dict(
    image_width = 64,
    image_height = 64,
    padding = 8,
)

model = train_from_prep(datagen_args, name='Image 64, padding 8',
                        preprocess_args=preprocess_args)

Preprocessing arguments changed. Performing data preprocessing...
